# Cited text spans identification

Finding the top x sentences in the body that have the largest similarity (as per ROUGE score) with the citations 

In [1]:
#Download the dataset
import requests
import io
import zipfile
import pandas as pd
import numpy as np
import os
# For visualizations
import matplotlib.pyplot as plt
# For regular expressions
import re
# For handling string
import string
# For performing mathematical operations
import math
# Importing spacy
import spacy
# Importing json to read input
import json
# Importing rouge for evaluation
from rouge_score import rouge_scorer

import xml.etree.ElementTree as ET
import seaborn as sns
import matplotlib.pyplot as plt
import html
from collections import defaultdict, Counter

import statistics as stats
import time

from scipy import spatial
from sent2vec.vectorizer import Vectorizer

# for turn text into sentences
import nltk.data
from nltk.corpus import wordnet as wn
from nltk.stem import WordNetLemmatizer 
from collections import Counter
from nltk.corpus import stopwords
import html
from lxml import etree
import unidecode

from scipy import stats as s


nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')


sns.set_style("dark")
plot_dims = (16, 16)


# url = "https://cs.stanford.edu/~myasu/projects/scisumm_net/scisummnet_release1.1__20190413.zip"
# response = requests.get(url)
# with zipfile.ZipFile(io.BytesIO(response.content)) as zipObj:
#     # Extract all the contents of zip file in different directory
#     zipObj.extractall("nlp_data")
#     print("File is unzipped in nlp_data folder") 



/opt/anaconda3/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/priscillaburity/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/priscillaburity/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/priscillaburity/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/priscillaburity/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
import nltk 
import glob, os
import re
import string
import sys
import random
import xml.etree.ElementTree as ET
import xml.etree
import numpy



In [3]:
# import pip
# from pip._internal import main as pipmain

# pipmain(['install', 'transformers'])
# pipmain(['install', 'SentencePiece'])

In [45]:
DATA_DIR = "../data/nlp_data/scisummnet_release1.1__20190413/top1000_complete"

In [5]:
#get all raw text, break all papers into two parts -- Abstract and rest of document
#first get all filepaths
xmlfiles = []
citations = []
for subdir, dirs, files in os.walk(DATA_DIR):
    for filename in files:
        filepath = subdir + os.sep + filename
        if filepath.endswith(".xml"):
            xmlfiles.append(filepath)
        if filepath.endswith(".json"):
            citations.append(filepath)

In [6]:
#next parse all XML documents

def parse_xml_abstract(fp):
    """ parse an XML journal article into an abstract and the rest of the text
    """
    try:
        tree = ET.parse(fp)
    except Exception as e:
        return "","",str(e)
    root = tree.getroot()
    
    ab = []
    bod = []
    
    for child in root:
        if child.tag == "ABSTRACT":
            for block in child:
                ab.append(block.text)
        else:
            for block in child:
                bod.append(block.text)
                
    #convert from list --> string
    abstract = "\n".join(ab)
    body = "\n".join(bod)
    
    #decode html entities
    abstract = html.unescape(abstract)
    body = html.unescape(body)
    
    return abstract,body,""




In [7]:
#create DF with papers and citations

raw_cols = []
for fpn in range(len(xmlfiles)):
    ab,bod,err = parse_xml_abstract(xmlfiles[fpn])
    if err:
        #print(fp, err)
        continue
    f = open(citations[fpn]) 

    # returns JSON object as  
    # a dictionary 
    data = json.load(f) 
    only_text = []
    for entry in data:
        only_text.append(entry['clean_text'])
#     print(only_text)
        
    raw_cols.append([ab,bod,only_text,xmlfiles[fpn]])

df = pd.DataFrame(raw_cols, columns=["abstract","body","citations", "filepath"])
df

,abstract,body,citations,filepath
0,We present a method for extracting parts of ob...,We present a method of extracting parts of obj...,[Berland and Charniak (1999) use Hearst style ...,/Users/priscillaburity/Documents/Berkeley/w266...
1,We describe a series of five statistical model...,We describe a series of five statistical model...,[The program takes the output of char_align (C...,/Users/priscillaburity/Documents/Berkeley/w266...
2,Previous work has shown that Chinese word segm...,Word segmentation is considered an important f...,[Chinese word segmentation is done by the Stan...,/Users/priscillaburity/Documents/Berkeley/w266...
3,We examine the viability of building large pol...,Polarity lexicons are large lists of phrases t...,[Recent work in this area includes Velikovich ...,/Users/priscillaburity/Documents/Berkeley/w266...
4,Extracting semantic relationships between enti...,Extraction of semantic relationships between e...,[They use two kinds of features: syntactic one...,/Users/priscillaburity/Documents/Berkeley/w266...
...,...,...,...,...
1004,"In statistical machine translation, correspond...","In statistical machine translation, correspond...","[In addition, Niessen and Ney (2004) decompose...",/Users/priscillaburity/Documents/Berkeley/w266...
1005,We have developed a new program called alignin...,Aligning parallel texts has recently received ...,[There have been quite a number of recent pape...,/Users/priscillaburity/Documents/Berkeley/w266...
1006,We present an approach to pronoun resolution b...,Pronoun resolution is a difficult but vital pa...,"[, We follow the closed track setting where sy...",/Users/priscillaburity/Documents/Berkeley/w266...
1007,We use logical inference techniques for recogn...,Recognising textual entailment (RTE) is the ta...,"[However, this method does not work for realwo...",/Users/priscillaburity/Documents/Berkeley/w266...


In [8]:
# NOT BEING USED

from dateutil.parser import parse

def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False

In [9]:
def checkQuality(sentence):
    
    ''' check the quality of body sentences, to make it eligible (or not) to be a cited text span'''
    
    lemmatizer = WordNetLemmatizer() 
    
    tokenized_text = nltk.word_tokenize(sentence)
    length = len(tokenized_text)
    alpha = 0
    nonAlpha = 0
    found = 0
    nonFound = 0
    upper = 0
    stop = 0
    stop_words = set(stopwords.words('english')) 
    
    for token in tokenized_text:
        #print(token)
        if len(token)==1:
            if token.isalpha():
                # count number of single alpha chars
                alpha+=1
            else:
                # count number of not single non-alpha chars
                nonAlpha+=1
        else:
            if(token.lower() in stop_words):
                # count number of stop words
                stop+=1
            else:
                lemma = lemmatizer.lemmatize(token.lower())
                # count number of Synsets for alpha tokens
                if token.isalpha():
                    if (len(wn.synsets(lemma))>0):
                        found+=1 # alpha tokens that have Synsets
                    else:
                        nonFound+=1 # alpha tokens that dont have Synsets
                else:
                    nonAlpha+=1 #non alpha tokens
    
    # calculate shares 
    alphaS = alpha/length
    nonAlphaS = nonAlpha/length
    foundS = found/length
    nonFoundS = nonFound/length
#     upperS = upper/length
    stopS = stop/length
    
    good_quality = True
    
    if(nonFoundS>0.2):
        good_quality = False
    if(foundS<0.1):
        good_quality = False
    if(alphaS>0.2):
        good_quality = False
    if(nonAlphaS>0.5):
        good_quality = False
    if len(tokenized_text)< 6:
        good_quality = False
    
    if ("equation" in sentence.lower()) | ("section" in sentence.lower()) | ("table" in sentence.lower()) | ("figure" in sentence.lower()) | ("=" in sentence) | (">" in sentence) | ("<" in sentence) | ("p(" in sentence.lower()):
        good_quality = False
    
    # remove sentences that has dates
    match = re.match(r'.*([1-3][0-9]{3})', sentence)
    if match is not None:
        good_quality = False 
        
    return good_quality



In [10]:
def cited_text_spans_ids(df,n_sent = 3):

    '''Find (n_sent) body sentences most similar to citations 
    inputs: 
        df: dataframe with simularity scores between each body sentence and citation 
        n_set: number of body sentences to output
    output:
        list of best ranked body sentetences ids
    
    ''' 
    
    # empty list where we'll store best scored body sentences 
    body_sent_all_ids = []
    body_sent_id = []
     
    # create an empty dataframe (num rows = num sentences in citations;  num col = n_sent)
    # we'll store here the body sentences ids with largest similarity
    sim_df_max = pd.DataFrame(0.0, index=[j for j in range(len(citations_sentences))], columns= [j for j in range(n_sent)])
    
    for ns in range(n_sent):
        # get the indexes that maximize similarity for each column (i.e., for each citation sentence)    
        sim_df_max[ns] = sim_df.idxmax(axis=0, skipna=True)
        # reset the largest values to zero, so we can retreive other top values in the loop
        sim_df[ns][sim_df_max[ns]] = 0

    
    # loop over the number of body sentences we want to retrieve
    for ns in range(n_sent): 
        # turn all columns into a list of best scored body sentences ids  
        li = sim_df_max[ns].tolist() 
        body_sent_all_ids = body_sent_all_ids + li
    
#     print(body_sent_all_ids)
    for ns in range(n_sent):   
        # append best scored sentence id over all citations sentences
        best_sent_id_single = int(s.mode(body_sent_all_ids)[0])
        body_sent_id.append(best_sent_id_single)
        # reset the largest values to zero, so we can retreive other top values in the loop                    
        body_sent_all_ids = list(filter(lambda a: a != best_sent_id_single, body_sent_all_ids)) 
#         print(body_sent_all_ids)
#         if len(body_sent_all_ids) == 0:
#             break

    return body_sent_id



In [42]:
# df['cited_text_spans'] = ""
# vectorizer = Vectorizer()
start = time.time()
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')
lemmatizer = WordNetLemmatizer() 
#instantiate rouge score
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

for paper_id in [i for i in df.index if i >79]:
    
    body_sentences = []
    # split body into sentences
    body_sentences_tok = df.body[paper_id].replace("et al.", "et al").replace("e.g.", "eg").replace("eg.", "eg").replace("i.e.", "ie").replace("ie.", "ie")
    body_sentences_tok = body_sentences_tok.replace(".1", ". ").replace(".2", ". ").replace(".3", ".").replace(".4", ". ").replace(".5", ". ").replace(".6", ". ").replace(".7", ". ").replace(".8", ". ").replace(".9", ". ")

    # tokenize body sentences
    body_sentences_tok = tokenizer.tokenize(body_sentences_tok)
    
    
    # truncate too large body sentences (larger than 512 throws an error - rare cases) and remove "?"
    body_sentences_tok = [s.replace("?", "") for s in body_sentences_tok if len(s) <= 512]
    
    # remove too small papers
    if len(body_sentences_tok) < 10:
        continue
        
    # get citations sentences (discarding empty entries)
    citations_sentences = list(filter(None, df.citations[paper_id]))
    
    # clean citations
    for c in range(len(citations_sentences)):
        citations_sentences[c] = re.sub(r"\s\([A-Z][a-z]+,\s[A-Z][a-z]?\.[^\)]*,\s\d{4}\)", "", citations_sentences[c])
    
    print(paper_id, "size after quality check:", len(body_sentences_tok))
    
    # truncate the center of the body for large bodies
    if len(body_sentences_tok) > 300:
        body_sentences_tok = body_sentences_tok[:150] + body_sentences_tok[-150:]
        print("truncated", paper_id, "new size:", len(body_sentences_tok))
        
    
    # find body sentences with quality
    for i in range(len(body_sentences_tok)):
        if len(body_sentences_tok[i])>0:
            if checkQuality(body_sentences_tok[i]):
                body_sentences.append(body_sentences_tok[i])
                
#     print("step 1 done")
    # create an empty dataframe (num rows = num sentences in body;  num col = num sentences in citations)
    sim_df = pd.DataFrame(0.0, index=[j for j in range(len(body_sentences))], columns=[j for j in range(len(citations_sentences))])

# #     # vectorize body sentences
# #     vectorizer.bert(body_sentences)
# #     vectors_bert_body = vectorizer.vectors

# #     # vectorize citations sentences
# #     vectorizer.bert(citations_sentences)
# #     vectors_bert_citations = vectorizer.vectors

#     print("step 2 done")

    # nested loop: over body sentences and citations sentences
    for body_sentence_id in range(len(body_sentences)):
        for citation_sentence_id in range(len(citations_sentences)):
            # fill empty dataframe with sim measure
#             sim_df[citation_sentence_id][body_sentence_id] = spatial.distance.cosine(vectors_bert_body[body_sentence_id], vectors_bert_citations[citation_sentence_id])
            scores = scorer.score(body_sentences[body_sentence_id],citations_sentences[citation_sentence_id] )
            sim_df[citation_sentence_id][body_sentence_id] = 100*scores['rouge2'][2]
#             print(body_sentence_id,citation_sentence_id,100*scores['rouge1'][2]  )
    
    
    
#     print("step 3 done")
    # get selected body sentences ids - if rouge 2 doesnt work, retry with rouge 1
    try:
        sent_ids = cited_text_spans_ids(sim_df,3)
    
    
    except: 
        for body_sentence_id in range(len(body_sentences)):
            for citation_sentence_id in range(len(citations_sentences)):
            # fill empty dataframe with sim measure
#             sim_df[citation_sentence_id][body_sentence_id] = spatial.distance.cosine(vectors_bert_body[body_sentence_id], vectors_bert_citations[citation_sentence_id])
                scores = scorer.score(body_sentences[body_sentence_id],citations_sentences[citation_sentence_id] )
                sim_df[citation_sentence_id][body_sentence_id] = 100*scores['rouge1'][2]
#             print(body_sentence_id,citation_sentence_id,100*scores['rouge1'][2]  )
        sent_ids = cited_text_spans_ids(sim_df,3)


        
        
    
#     print("step 4 done")
    # fill our large/original dataframe with the best scored body sentences 
    df.cited_text_spans[paper_id] = [body_sentences[b] for b in sent_ids]
    
    if paper_id % 1 == 0:
        print(paper_id, "secs to run:", time.time() - start)
        start = time.time()



80 size after quality check: 176
80 secs to run: 2.5009191036224365
81 size after quality check: 355
truncated 81 new size: 300
81 secs to run: 2.801462173461914
82 size after quality check: 137
82 secs to run: 1.3253841400146484
83 size after quality check: 148
83 secs to run: 2.443950653076172
84 size after quality check: 230
84 secs to run: 3.2808339595794678
85 size after quality check: 107
85 secs to run: 0.9367637634277344
86 size after quality check: 266
86 secs to run: 3.416733741760254
87 size after quality check: 209
87 secs to run: 3.7769811153411865
88 size after quality check: 161
88 secs to run: 1.1530570983886719
90 size after quality check: 180
90 secs to run: 2.386312961578369
91 size after quality check: 79
91 secs to run: 0.8459310531616211
92 size after quality check: 268
92 secs to run: 3.1421468257904053
93 size after quality check: 275
93 secs to run: 1.8849501609802246
94 size after quality check: 184
94 secs to run: 2.5000040531158447
95 size after quality chec

199 secs to run: 3.805853843688965
200 size after quality check: 322
truncated 200 new size: 300
200 secs to run: 4.801187038421631
202 size after quality check: 156
202 secs to run: 3.9615259170532227
203 size after quality check: 251
203 secs to run: 2.7600460052490234
204 size after quality check: 162
204 secs to run: 1.7903599739074707
205 size after quality check: 128
205 secs to run: 2.734579086303711
206 size after quality check: 31
206 secs to run: 0.5536060333251953
207 size after quality check: 170
207 secs to run: 2.475337028503418
208 size after quality check: 157
208 secs to run: 1.5101900100708008
209 size after quality check: 233
209 secs to run: 2.6665399074554443
210 size after quality check: 73
210 secs to run: 1.075416088104248
211 size after quality check: 200
211 secs to run: 2.9556758403778076
212 size after quality check: 157
212 secs to run: 2.4625020027160645
213 size after quality check: 175
213 secs to run: 2.7361409664154053
214 size after quality check: 302

320 secs to run: 0.9918391704559326
321 size after quality check: 519
truncated 321 new size: 300
321 secs to run: 5.95051121711731
322 size after quality check: 186
322 secs to run: 3.626850128173828
323 size after quality check: 313
truncated 323 new size: 300
323 secs to run: 2.713013172149658
324 size after quality check: 114
324 secs to run: 2.018272876739502
325 size after quality check: 163
325 secs to run: 2.896204948425293
326 size after quality check: 199
326 secs to run: 2.0785300731658936
327 size after quality check: 44
327 secs to run: 0.2545197010040283
328 size after quality check: 318
truncated 328 new size: 300
328 secs to run: 4.888899803161621
330 size after quality check: 162
330 secs to run: 2.867522954940796
331 size after quality check: 208
331 secs to run: 3.5524909496307373
332 size after quality check: 132
332 secs to run: 3.2083961963653564
333 size after quality check: 178
333 secs to run: 2.358186960220337
334 size after quality check: 159
334 secs to run:

438 secs to run: 6.846342325210571
439 size after quality check: 178
439 secs to run: 3.126620292663574
440 size after quality check: 209
440 secs to run: 3.0364999771118164
441 size after quality check: 396
truncated 441 new size: 300
441 secs to run: 1.8848068714141846
442 size after quality check: 224
442 secs to run: 4.797733783721924
443 size after quality check: 121
443 secs to run: 2.434561014175415
444 size after quality check: 198
444 secs to run: 0.9572720527648926
445 size after quality check: 173
445 secs to run: 1.6936447620391846
446 size after quality check: 229
446 secs to run: 4.487959861755371
447 size after quality check: 154
447 secs to run: 1.2987151145935059
448 size after quality check: 296
448 secs to run: 3.2031960487365723
449 size after quality check: 652
truncated 449 new size: 300
449 secs to run: 5.858459949493408
450 size after quality check: 121
450 secs to run: 2.6781692504882812
451 size after quality check: 185
451 secs to run: 2.99043607711792
452 si

558 secs to run: 3.1418869495391846
559 size after quality check: 199
559 secs to run: 1.979583978652954
560 size after quality check: 149
560 secs to run: 3.908431053161621
561 size after quality check: 182
561 secs to run: 3.983307123184204
563 size after quality check: 159
563 secs to run: 1.6525380611419678
564 size after quality check: 209
564 secs to run: 4.215935945510864
565 size after quality check: 184
565 secs to run: 1.110260009765625
566 size after quality check: 179
566 secs to run: 2.5449609756469727
567 size after quality check: 219
567 secs to run: 2.3748631477355957
568 size after quality check: 172
568 secs to run: 3.205925226211548
569 size after quality check: 200
569 secs to run: 3.408123016357422
570 size after quality check: 196
570 secs to run: 4.714776039123535
572 size after quality check: 61
572 secs to run: 0.6420989036560059
573 size after quality check: 101
573 secs to run: 1.6993718147277832
574 size after quality check: 162
574 secs to run: 1.5997061729

682 secs to run: 2.684657096862793
683 size after quality check: 159
683 secs to run: 2.6005148887634277
685 size after quality check: 380
truncated 685 new size: 300
685 secs to run: 3.821565866470337
686 size after quality check: 148
686 secs to run: 3.0734548568725586
688 size after quality check: 658
truncated 688 new size: 300
688 secs to run: 5.750694036483765
689 size after quality check: 202
689 secs to run: 2.62925386428833
690 size after quality check: 365
truncated 690 new size: 300
690 secs to run: 5.9278881549835205
691 size after quality check: 161
691 secs to run: 2.5858590602874756
692 size after quality check: 118
692 secs to run: 2.6861610412597656
693 size after quality check: 162
693 secs to run: 1.8102121353149414
694 size after quality check: 521
truncated 694 new size: 300
694 secs to run: 2.15710711479187
695 size after quality check: 146
695 secs to run: 3.3580782413482666
697 size after quality check: 108
697 secs to run: 1.6862740516662598
698 size after qual

800 secs to run: 5.925928115844727
801 size after quality check: 152
801 secs to run: 246.60036206245422
802 size after quality check: 181
802 secs to run: 3.5130701065063477
805 size after quality check: 196
805 secs to run: 3.202153205871582
806 size after quality check: 160
806 secs to run: 3.2845780849456787
807 size after quality check: 195
807 secs to run: 2.6357169151306152
808 size after quality check: 190
808 secs to run: 2.8705925941467285
809 size after quality check: 208
809 secs to run: 3.992096185684204
810 size after quality check: 193
810 secs to run: 3.696942090988159
811 size after quality check: 224
811 secs to run: 4.647194862365723
812 size after quality check: 169
812 secs to run: 3.0119569301605225
813 size after quality check: 178
813 secs to run: 2.567103862762451
814 size after quality check: 163
814 secs to run: 2.593411922454834
815 size after quality check: 205
815 secs to run: 2.6041719913482666
816 size after quality check: 141
816 secs to run: 0.85408306

922 secs to run: 5.432880163192749
923 size after quality check: 81
923 secs to run: 1.9257831573486328
924 size after quality check: 515
truncated 924 new size: 300
924 secs to run: 5.3069212436676025
926 size after quality check: 170
926 secs to run: 3.4643778800964355
927 size after quality check: 168
927 secs to run: 1.512289047241211
928 size after quality check: 145
928 secs to run: 2.4083571434020996
929 size after quality check: 227
929 secs to run: 3.351301908493042
930 size after quality check: 421
truncated 930 new size: 300
930 secs to run: 2.7167751789093018
931 size after quality check: 111
931 secs to run: 1.3287479877471924
932 size after quality check: 213
932 secs to run: 3.6977670192718506
933 size after quality check: 159
933 secs to run: 2.264461040496826
934 size after quality check: 94
934 secs to run: 1.2244863510131836
935 size after quality check: 65
935 secs to run: 1.3403148651123047
936 size after quality check: 160
936 secs to run: 1.7575418949127197
937 s

In [39]:
sim_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,0.000000,0.0,0.000000,14.492754,3.333333,10.526316,18.918919,12.987013,0.000000,10.344828,9.375000,4.255319,3.030303,8.695652,3.703704,8.333333,12.987013,8.333333
1,6.896552,0.0,0.000000,5.405405,7.142857,4.545455,4.761905,4.444444,3.571429,7.692308,6.250000,0.000000,5.882353,5.405405,9.090909,5.000000,4.444444,5.000000
2,6.666667,0.0,3.636364,5.263158,0.000000,0.000000,0.000000,4.347826,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,4.878049
3,0.000000,20.0,0.000000,14.634146,6.250000,8.333333,13.043478,12.244898,6.666667,20.000000,11.111111,10.526316,5.263158,19.512195,15.384615,9.090909,12.244898,9.090909
4,0.000000,0.0,0.000000,5.714286,0.000000,4.761905,5.000000,4.651163,0.000000,8.333333,6.666667,15.384615,0.000000,5.714286,0.000000,5.263158,4.651163,5.263158
5,0.000000,0.0,0.000000,15.000000,6.451613,8.510638,13.333333,12.500000,6.779661,20.689655,11.428571,11.111111,5.405405,20.000000,16.000000,9.302326,12.500000,9.302326
6,0.000000,0.0,0.000000,8.163265,10.000000,14.285714,14.814815,14.035088,5.882353,15.789474,4.545455,0.000000,17.391304,8.163265,5.882353,7.692308,14.035088,15.384615
7,0.000000,0.0,0.000000,4.651163,0.000000,16.000000,16.666667,3.921569,6.451613,18.750000,5.263158,0.000000,20.000000,23.255814,21.428571,8.695652,7.843137,8.695652
8,0.000000,0.0,0.000000,10.000000,0.000000,12.765957,17.777778,8.333333,0.000000,13.793103,11.428571,11.111111,5.405405,15.000000,0.000000,13.953488,4.166667,9.302326


In [29]:
body_sentences_tok

['A working definition of coreference resolution is partitioning the noun phrases we are interested in into equiv alence classes, each of which refers to a physical entity.We adopt the terminologies used in the Automatic Con tent Extraction (ACE) task (NIST, 2003a) and call eachindividual phrase a mention and equivalence class an en tity.',
 'For example, in the following text segment, (1): ?The American Medical Association voted yesterday to install the heir apparent as its president-elect, rejecting a strong, upstart challenge by a district doctor who argued that the nation?s largest physicians?',
 'group needs stronger ethics and new leadership.',
 '?mentions are underlined, ?American Medical Associa tion?, ?its?',
 'and ?group?',
 'refer to the same organization(object) and they form an entity.',
 'Similarly, ?the heir ap parent?',
 'and ?president-elect?',
 'refer to the same person and they form another entity.',
 'It is worth pointing out that the entity definition here is diffe

In [44]:

print(df.cited_text_spans[80])
print('------')
# df.citations[50]
list(filter(None, df.citations[80]))


['The work was motivated by the needs of the ILEX system for generating descriptions of museum artefacts (in particular, 20th Century jewellery) [Mellish et al 98].', 'This paper presents some initial experiments using stochastic search methods for aspects of text planning.', 'In this task, one is given a set of facts all of which should be included in a text and a set of relations between facts, some of which can be included in the text.']
------


['Mellish et al (1998) investigate the problem of determining a discourse tree for a set of elementary speech acts which are partially constrained by rhetorical relations.',
 'Mellish et al (1998) (and subsequently Karamanis and Manurung 2002) advocate genetic algorithms as an alternative to exhaustively searching for the optimal ordering of descriptions of museum artefacts.',
 'Following previous work (Mellish et al, 1998) we used a single fitness function that scored candidates based on their coherence.',
 'The genetic algorithms of Mellish et al (1998) and Karamanis and Manarung (2002), as well as the greedy algorithm of Lapata (2003), provide no theoretical guarantees on the optimality of the solutions they propose.',
 'For example, the measure from (Mellish et al, 1998) looks at the entire discourse up to the current transition for some of their cost factors.',
 'Mellish et al (1998) advocate stochastic search as an alternative to exhaustively examining the search space.',
 'As in

In [ ]:
# import multiprocessing as mp
# from pathos.multiprocessing import ProcessingPool as Pool


In [ ]:
# #in parallel

# start = time.time()
# df['cited_text_spans'] = ""

# def cited_text_spans(df):
#     start = time.time()
#     vectorizer = Vectorizer()
    
#     for paper_id in range(2):#df.index:

#         body_sentences = df.body[paper_id].replace("\n","").split(".")
#         citations_sentences = df.citations[paper_id]

#         cosine_df = pd.DataFrame(0.0, index=[i for i in range(len(body_sentences))], columns=[i for i in range(len(citations_sentences))])

#         vectorizer.bert(body_sentences)
#         vectors_bert_body = vectorizer.vectors

#         vectorizer.bert(citations_sentences)
#         vectors_bert_citations = vectorizer.vectors

#         for body_sentence_id in range(len(body_sentences)):
#             for citation_sentence_id in range(len(citations_sentences)):
#                 cosine_df[citation_sentence_id][body_sentence_id] = spatial.distance.cosine(vectors_bert_body[body_sentence_id], vectors_bert_citations[citation_sentence_id])


#         sent_ids = cited_text_spans_ids(cosine_df,2)

#         df.cited_text_spans[paper_id] = [body_sentences[b] for b in sent_ids]

#         if paper_id % 1 == 0:
#             print(paper_id, time.time() - start)
#             start = time.time()



In [ ]:
# cores=mp.cpu_count()-6

# df_split = np.array_split(df, cores, axis=0)

# # create the multiprocessing pool
# pool = Pool(cores)

# # process the DataFrame by mapping function to each df across the pool
# df_out = np.vstack(pool.map(cited_text_spans, df_split))

# # close down the pool and join
# pool.close()
# pool.join()
# pool.clear()

# df_out